# データの読み込み


In [1]:
# 必要なライブラリをインポート
import os
import pandas as pd

In [ ]:
# 分野とカテゴリを指定
fields = ["Physics", "Biochemistry_Molecular_Biology", "Chemistry", "Materials_Science", "Engineering"]
categories = ["high", "low"]

# 入力ファイルパスと出力ファイルパスのベースディレクトリ
input_dir = "../../data/results"


In [4]:
# データを格納する辞書を用意
dataframes = {}

# データの読み込み
for field in fields:
    for category in categories:
        # ファイルパスを生成
        file_path = os.path.join(input_dir, f"{field}_{category}1000.csv")
        
        # ファイルが存在するかを確認
        if os.path.exists(file_path):
            try:
                # データフレームを読み込む
                df = pd.read_csv(file_path, encoding="utf-8")
                key = f"{field}_{category}"
                dataframes[key] = df
                print(f"ファイル {file_path} を読み込みました。")
            except Exception as e:
                print(f"ファイル {file_path} の読み込み中にエラーが発生しました: {e}")
        else:
            print(f"ファイル {file_path} が存在しません。")


ファイル ../data/results/Physics_high1000.csv を読み込みました。
ファイル ../data/results/Physics_low1000.csv を読み込みました。
ファイル ../data/results/Biochemistry_Molecular_Biology_high1000.csv を読み込みました。
ファイル ../data/results/Biochemistry_Molecular_Biology_low1000.csv を読み込みました。
ファイル ../data/results/Chemistry_high1000.csv を読み込みました。
ファイル ../data/results/Chemistry_low1000.csv を読み込みました。
ファイル ../data/results/Materials_Science_high1000.csv を読み込みました。
ファイル ../data/results/Materials_Science_low1000.csv を読み込みました。
ファイル ../data/results/Engineering_high1000.csv を読み込みました。
ファイル ../data/results/Engineering_low1000.csv を読み込みました。


# データの前処理


In [14]:
# 変換対象のruleカラムのリストを定義
rule_columns = [f"rule{i}" for i in range(1, 35)]

# yes -> 1, no -> 0 の変換関数
def transform_rules(df, rule_cols):
    for col in rule_cols:
        if col in df.columns:
            df[col] = df[col].replace({'yes': 1, 'no': 0})
    return df

# 各データフレームに対して変換処理を実行
for key, df in dataframes.items():
    dataframes[key] = transform_rules(df, rule_columns)
    print(f"{key} のデータを変換しました。")

# 変換結果の確認
for key, df in dataframes.items():
    print(f"{key} データフレームの先頭5行:")
    print(df.head())


/var/folders/kb/fcmv44k11vvczn4k2qxly51c0000gn/T/ipykernel_22997/3125639715.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'yes': 1, 'no': 0})


Physics_high のデータを変換しました。
Physics_low のデータを変換しました。
Biochemistry_Molecular_Biology_high のデータを変換しました。
Biochemistry_Molecular_Biology_low のデータを変換しました。
Chemistry_high のデータを変換しました。
Chemistry_low のデータを変換しました。
Materials_Science_high のデータを変換しました。
Materials_Science_low のデータを変換しました。
Engineering_high のデータを変換しました。
Engineering_low のデータを変換しました。
Physics_high データフレームの先頭5行:
   ID                                           Abstract  rule1  rule2  rule3  \
0   1  Graphene is a rapidly rising star on the horiz...    1.0      0      0   
1   2  The method of dispersion correction as an add-...    1.0      0      0   
2   3  Gaussian basis sets of quadruple zeta valence ...    1.0      0      0   
3   4  A modification of the nudged elastic band meth...    0.0  happy  happy   
4   5  Topological insulators are electronic material...    1.0      0      0   

   rule4  rule5  rule6  rule7  rule8  ... rule25 rule26  rule27  rule28  \
0    0.0    0.0      0      1  happy  ...    1.0    1.0       1   happy   
1  

# データの確認


In [15]:
for key, df in dataframes.items():
    print(f"{key} データフレームの先頭5行:")
    print(df.head())

Physics_high データフレームの先頭5行:
   ID                                           Abstract  rule1  rule2  rule3  \
0   1  Graphene is a rapidly rising star on the horiz...    1.0      0      0   
1   2  The method of dispersion correction as an add-...    1.0      0      0   
2   3  Gaussian basis sets of quadruple zeta valence ...    1.0      0      0   
3   4  A modification of the nudged elastic band meth...    0.0  happy  happy   
4   5  Topological insulators are electronic material...    1.0      0      0   

   rule4  rule5  rule6  rule7  rule8  ... rule25 rule26  rule27  rule28  \
0    0.0    0.0      0      1  happy  ...    1.0    1.0       1   happy   
1    1.0    0.0      0      0      1  ...    1.0    1.0       1       1   
2    1.0    1.0      0      1      0  ...    1.0    1.0       0       1   
3    0.0    0.0  happy  happy  happy  ...    1.0    1.0   happy   happy   
4    1.0    0.0      0      0      0  ...    1.0    1.0       1   happy   

   rule29  rule30  rule31  rule32  